In [54]:
import re
import csv
import string
import numpy as np
import pandas as pd
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import PorterStemmer
from stop_words import safe_get_stop_words
from stop_words import get_stop_words
stop_words = get_stop_words('english')
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
PorterStemmer().stem('complications')
stemmer = PorterStemmer()
from nltk import everygrams
from nltk import FreqDist

# 1- Loading data

In [55]:
data_ =pd.read_csv("AllCoursesDB.csv",sep=',',encoding = "ISO-8859-1")
data_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
Course_Title      1415 non-null object
Course_type       1415 non-null object
Course_Skills     1415 non-null object
Course_Descrip    1415 non-null object
Course_Link       1415 non-null object
Keyword_Link      1415 non-null object
web_site          1415 non-null object
dtypes: object(7)
memory usage: 77.5+ KB


### - Eliminate Duplicate Items by Product Category and Product Link

In [56]:
# Eliminate Duplicate Items by Product Category and Product Link 
#toclean = pd.read_csv("AllCoursesDB.csv", sep=',', encoding="ISO-8859-1")
#deduped = toclean.drop_duplicates(subset=['Jop_Link'], keep='first')
#deduped.to_csv('DBCourses.csv', sep=',', encoding="ISO-8859-1")

In [57]:
data =pd.read_csv("DBCourses.csv",sep=',',encoding = "ISO-8859-1")
data.head()

,EL,Course_Title,Course_type,Course_Skills,Course_Descrip,Course_Link,Keyword_Link,web_site
0,NaN,Introduction to Big Data,COURSE,"Big Data , Apache Hadoop , Mapreduce , Cloudera",Interested in increasing your knowledge of the...,https://www.coursera.org/learn/big-data-introd...,https://www.coursera.org/search?query=Big Data,coursera
1,NaN,Google Cloud Platform Big Data and Machine Lea...,COURSE,"Tensorflow , Bigquery , Google Cloud Platform ...",This 2-week accelerated on-demand course intro...,https://www.coursera.org/learn/gcp-big-data-ml...,https://www.coursera.org/search?query=Big Data,coursera
2,NaN,Scalable Machine Learning on Big Data using Ap...,COURSE,"Data Science , Artificial Intelligence (AI) , ...",This course will empower you with the skills t...,https://www.coursera.org/learn/machine-learnin...,https://www.coursera.org/search?query=Big Data,coursera
3,NaN,IBM AI Engineering,PROFESSIONAL CERTIFICATE,"Data Science , Deep Learning , Artificial Inte...",The rapid pace of innovation in Artificial Int...,https://www.coursera.org/professional-certific...,https://www.coursera.org/search?query=Big Data,coursera
4,NaN,"Big Data Analysis: Hive, Spark SQL, DataFrames...",COURSE,"Graphs , Hive , Apache Hive , Apache Spark",No doubt working with huge data volumes is har...,https://www.coursera.org/learn/big-data-analysis,https://www.coursera.org/search?query=Big Data,coursera


In [58]:
data=data[data["EL"]!=1]
#data.info()

# 2- DATA PREPROCESSING 

In [59]:
def count_words(x):
    mapping =dict.fromkeys(map(ord,string.punctuation))
    words = {}
    for m in x:
        try:
            sent=m.split(' ')
        except:
            pass
        for word in sent:
            word=word.strip()
            word=word.lower()
            word=word.translate(mapping)
            if word in stop_words:
                continue
            if len(word)<3:         
                continue
            if word in words:
                words[word]+=1
            else:
                words[word]=1
    print('done')
    return words

In [60]:
words=count_words(data["Course_Skills"])
df_words = pd.DataFrame(list(words.items()),columns=['word','count'])
df_words_max=df_words.nlargest(25, 'count')  # to show 25 max count words
df_words_max

done


,word,count
1,data,298
37,programming,182
23,analysis,150
16,learning,126
969,free,101
39,python,93
15,machine,82
44,management,68
102,marketing,56
107,statistics,46


In [61]:
# List of words to be deleted
df_words =pd.read_csv("df_words.csv",sep=',',encoding = "ISO-8859-1")
df_remove_words=df_words[df_words["dl"]==1]
df_remove_words[1:10]

,Unnamed: 0,word,count,dl
15,8,google,40,1.0
17,489,new,38,1.0
23,136,technology,31,1.0
24,103,network,30,1.0
31,961,institute,27,1.0
33,989,georgia,26,1.0
35,93,software,25,1.0
36,90,information,23,1.0
39,309,testing,22,1.0


In [62]:
# add new stop words
stopwords_=df_remove_words["word"].values.tolist()

In [63]:
# Remove the added stopwords
def remove_stopwords(tx):
    query = tx
    querywords = query.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords_]
    result = ' '.join(resultwords)
    return result

In [64]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

In [65]:
data["Course_Skills"]=data["Course_Skills"].astype(str).apply(lambda x: x.lower())

data["Course_Skills"]=data["Course_Skills"].apply(strip_multiple_whitespaces) 

data["Course_Skills"]=data["Course_Skills"].apply(lambda x: x.strip())

data["Course_Skills"]=data["Course_Skills"].apply(lambda x: re.sub(r"\d+", "", x))

In [66]:
data["Course_Skills"][10]

'data model , big data , data modeling , data management'

In [67]:
#data.to_csv('C:/Users/moh_e/Desktop/dfdataafterNgrams_df_.csv') 

In [68]:
cleaned_data=data["Course_Skills"].astype(str).apply(lambda x: x.lower())            # Convert text to lowercase
cleaned_data=cleaned_data.apply(strip_non_alphanum)                                  # Remove strip_non_alphanum 
cleaned_data=cleaned_data.apply(lambda x: re.sub(r"\d+", "", x))                     # Remove numbers 
cleaned_data=cleaned_data.apply(strip_punctuation)                                   # Remove punctuation   
cleaned_data=cleaned_data.apply(lambda x: strip_short(x,minsize=3))                  # Remove short words
cleaned_data=cleaned_data.apply(strip_tags)                                          # Remove tags
data["Course_Skills"]=cleaned_data

In [69]:
cleaned_data=data["Course_Skills"].apply(remove_stopwords)                        # Remove the added stopwords
data["Course_Skills"]=cleaned_data

In [70]:
data["Course_Skills"][10]

'data model big data data modeling data management'

In [71]:
#data.to_csv('C:/Users/moh_e/Desktop/datas55.csv') 

In [72]:
processed_data=data["Course_Skills"].astype(str).apply(preprocess)
processed_data[10]

['data', 'model', 'big', 'data', 'data', 'model', 'data', 'manag']

In [73]:
processed_data = list(map(lambda line: list(filter(lambda word: word not in stop_words, line)), processed_data))

# 3- Preparing for LDA

In [74]:
# Create Dictionary
import gensim.corpora as corpora
id2word = corpora.Dictionary(processed_data)

In [75]:
# Create Corpus
corpus = [id2word.doc2bow(text) for text in processed_data]

# 4- LDA model training and results visualization

In [76]:
# LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [77]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic)) 

Topic: 0 
Words: 0.217*"learn" + 0.134*"machin" + 0.068*"intellig" + 0.058*"artifici" + 0.046*"deep" + 0.037*"optim" + 0.032*"bayesian" + 0.030*"project" + 0.030*"probabl" + 0.030*"scienc"
Topic: 1 
Words: 0.094*"plan" + 0.085*"cloud" + 0.071*"javascript" + 0.064*"comput" + 0.062*"tensorflow" + 0.058*"css" + 0.043*"html" + 0.028*"strateg" + 0.025*"platform" + 0.025*"api"
Topic: 2 
Words: 0.268*"data" + 0.122*"analysi" + 0.094*"model" + 0.031*"structur" + 0.030*"busi" + 0.027*"analyt" + 0.024*"excel" + 0.024*"financi" + 0.023*"cluster" + 0.021*"alteryx"
Topic: 3 
Words: 0.115*"data" + 0.094*"statist" + 0.060*"process" + 0.058*"analysi" + 0.058*"regress" + 0.057*"visual" + 0.040*"program" + 0.038*"linear" + 0.038*"distribut" + 0.031*"dataviz"
Topic: 4 
Words: 0.152*"manag" + 0.139*"design" + 0.134*"market" + 0.042*"analyt" + 0.042*"code" + 0.040*"neural" + 0.026*"digit" + 0.023*"custom" + 0.023*"match" + 0.022*"financ"
Topic: 5 
Words: 0.276*"program" + 0.149*"python" + 0.085*"algorithm"

In [91]:
# Compute Perplexity Score
#print('\nPerplexity Score: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=processed_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

In [79]:
num_topics=6

def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 30);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [80]:
Topics=get_lda_topics(lda_model, num_topics)

In [81]:
Topics

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06
0,learn,plan,data,data,manag,program
1,machin,cloud,analysi,statist,design,python
2,intellig,javascript,model,process,market,algorithm
3,artifici,comput,structur,analysi,analyt,languag
4,deep,tensorflow,busi,regress,code,java
5,optim,css,analyt,visual,neural,databas
6,bayesian,html,excel,program,digit,secur
7,project,strateg,financi,linear,custom,social
8,probabl,platform,cluster,distribut,match,cryptographi
9,scienc,api,alteryx,dataviz,financ,orient


In [82]:
#Topics.to_csv('C:/Users/moh_e/Desktop/MOOCS6topics_.csv') 

In [83]:
# Visualize the topics
#import pyLDAvis
#import pyLDAvis.gensim 
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

In [84]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='tsne')
vis


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
3      23.337934        1       1  190.455917 -103.906944
2      22.506474        1       2  -82.426704   77.597641
0      17.058557        1       3  119.949486   85.970291
5      14.807538        1       4   31.660269 -229.641708
4      12.080561        1       5   24.524220  -57.490490
1      10.208936        1       6 -136.996750 -117.467934, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
2     Default  267.000000          data  267.000000  30.0000  30.0000
27    Default  154.000000       program  154.000000  29.0000  29.0000
12    Default  113.000000         learn  113.000000  28.0000  28.0000
13    Default   70.000000        machin   70.000000  27.0000  27.0000
29    Default   85.000000        python   85.000000  26.0000  26.0000
30    Default   57.000000         manag   57.000000  25.0000  25.0000
102   Default   51.000000        design   51.000000  24.0000  24.0000
64    Default   50.000000        market   50.000000  23.0000  23.0000
17    Default  126.000000       analysi  126.000000  22.0000  22.0000
68    Default   67.000000       statist   67.000000  21.0000  21.0000
31    Default   65.000000         model   65.000000  20.0000  20.0000
44    Default   49.000000     algorithm   49.000000  19.0000  19.0000
80    Default   30.000000          plan   30.000000  18.0000  18.0000
6     Default   27.000000         cloud   27.000000  17.0000  17.0000
11    Default   36.000000      intellig   36.000000  16.0000  16.0000
58    Default   43.000000       process   43.000000  15.0000  15.0000
74    Default   42.000000       regress   42.000000  14.0000  14.0000
22    Default   41.000000        visual   41.000000  13.0000  13.0000
10    Default   31.000000      artifici   31.000000  12.0000  12.0000
208   Default   22.000000    javascript   22.000000  11.0000  11.0000
95    Default   26.000000       languag   26.000000  10.0000  10.0000
7     Default   20.000000        comput   20.000000   9.0000   9.0000
9     Default   20.000000    tensorflow   20.000000   8.0000   8.0000
207   Default   18.000000           css   18.000000   7.0000   7.0000
16    Default   24.000000          deep   24.000000   6.0000   6.0000
37    Default   35.000000        analyt   35.000000   5.0000   5.0000
34    Default   28.000000        linear   28.000000   4.0000   4.0000
49    Default   27.000000     distribut   27.000000   3.0000   3.0000
144   Default   18.000000          java   18.000000   2.0000   2.0000
113   Default   20.000000         optim   20.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
80     Topic6   29.192339          plan   30.029496   2.2536  -2.3684
6      Topic6   26.553332         cloud   27.384374   2.2511  -2.4631
208    Topic6   22.165767    javascript   22.997582   2.2451  -2.6438
7      Topic6   19.966511        comput   20.797749   2.2411  -2.7482
9      Topic6   19.358714    tensorflow   20.192159   2.2398  -2.7792
207    Topic6   18.132375           css   18.964542   2.2370  -2.8446
205    Topic6   13.528201          html   14.359215   2.2223  -3.1375
81     Topic6    8.846981       strateg    9.681386   2.1918  -3.5622
8      Topic6    7.786074      platform    8.616682   2.1805  -3.6900
62     Topic6    7.659807           api    8.492221   2.1787  -3.7063
63     Topic6    7.649952      interfac    8.483934   2.1784  -3.7076
268    Topic6    6.433578       sequenc    7.264585   2.1604  -3.8808
267    Topic6    6.433578         async    7.264585   2.1604  -3.8808
79     Topic6    6.391423    leadership    7.224636   2.1594  -3.8873
5      Topic6    5.922490      bigqueri    6.753575   2.1506  -3.9635
187    Topic6    5.390512            aw    6.228917   2.1373  -4.0577
210    Topic6    3.828395        jqueri    4.658842   2.0856  -

In [85]:
#pyLDAvis.save_html(vis, 'C:/Users/moh_e/Desktop/Nurturing Data Talent_2/Fainal/Visualize topics (courses).html')

In [86]:
#import pandas as pd

#top_words_per_topic = []
#for t in range(lda_model.num_topics):
#    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 30)])

#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("C:/Users/moh_e/Desktop/top_words(Courses).csv")

## - Finding topics for each document

In [87]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [88]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=processed_data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.8550,"data, statist, process, analysi, regress, visu...","[big, data, apach, hadoop, mapreduc]"
1,1,1.0,0.8625,"plan, cloud, javascript, comput, tensorflow, c...","[tensorflow, bigqueri, cloud, platform, cloud,..."
2,2,0.0,0.4944,"learn, machin, intellig, artifici, deep, optim...","[data, scienc, artifici, intellig, machin, lea..."
3,3,0.0,0.6363,"learn, machin, intellig, artifici, deep, optim...","[data, scienc, deep, learn, artifici, intellig..."
4,4,3.0,0.7874,"data, statist, process, analysi, regress, visu...","[apach, apach, spark]"
5,5,2.0,0.6935,"data, analysi, model, structur, busi, analyt, ...","[data, inform, decis, make, big, data, data, a..."
6,6,2.0,0.8599,"data, analysi, model, structur, busi, analyt, ...","[consupt, base, data, mine, scritpt]"
7,7,3.0,0.8746,"data, statist, process, analysi, regress, visu...","[program, big, data, apach, spark, sql]"
8,8,0.0,0.5152,"learn, machin, intellig, artifici, deep, optim...","[python, program, machin, learn, machin, learn..."
9,9,3.0,0.6119,"data, statist, process, analysi, regress, visu...","[apach, apach, data, analysi, big, data, sql]"


In [89]:
#df_dominant_topic.to_csv('C:/Users/moh_e/Desktop/_dominant_6topics.csv')

## - The most representative document for each topic

In [90]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,0.8855,"learn, machin, intellig, artifici, deep, optim, bayesian, project, probabl, scienc","[machin, learn, deep, learn, multi, task, learn]"
1,1.0,0.8625,"plan, cloud, javascript, comput, tensorflow, css, html, strateg, platform, api","[tensorflow, bigqueri, cloud, platform, cloud, comput]"
2,2.0,0.9226,"data, analysi, model, structur, busi, analyt, excel, financi, cluster, alteryx","[data, scienc, busi, analyt, decis, make, data, analysi, big, data]"
3,3.0,0.9121,"data, statist, process, analysi, regress, visual, program, linear, distribut, dataviz","[python, program, data, visual, dataviz, linear, regress, sql, data]"
4,4.0,0.8965,"manag, design, market, analyt, code, neural, digit, custom, match, financ","[digit, market, market, analyt, analyt, market, perform, manag]"
5,5.0,0.8847,"program, python, algorithm, languag, java, databas, secur, social, cryptographi, orient","[program, java, program, orient, program, oop, algorithm]"
